In [5]:
import requests, pandas as pd, time
from newspaper import Article
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
from tqdm import tqdm

BASE_URL = "https://www.hankyung.com/article/"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/127.0.0.0 Safari/537.36"),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
    "Referer": "https://www.hankyung.com/",
}

def crawl_url(url):
    """단일 URL 크롤링: AMP 먼저 요청, 없으면 일반 페이지 시도"""
    try:
        # 1. AMP 페이지 먼저 시도
        amp = url.rstrip("/") + "/amp"
        r = requests.get(amp, headers=HEADERS, timeout=12)

        # 2. AMP 없으면 일반 페이지 시도
        if r.status_code == 404:
            r = requests.get(url, headers=HEADERS, timeout=12)

        # 3. 여전히 에러면 None
        if r.status_code in (403, 404):
            return None
        r.raise_for_status()

        # 4. newspaper3k 파싱
        art = Article(url, language='ko')
        art.set_html(r.text)
        art.parse()
        if not art.title or not art.text:
            return None

        return {
            "url": url,
            "title": art.title,
            "text": art.text,
            "publish_date": art.publish_date
        }
    except Exception:
        return None

def crawl_gen_articles_range(date_str, start_id, end_id, *,  # end_id는 '미포함' (파이썬 range 규칙)
                             max_workers=36, batch_size=1000, sleep_between=0.05,
                             save_path=None):
    """
    일반 기사 크롤링: [start_id, end_id) 구간만
      - AMP만 요청
      - 배치 제출
    """
    results, collected = [], 0
    total = end_id - start_id

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        with tqdm(total=total, desc=f"{date_str} [{start_id:05}-{end_id-1:05}]") as pbar:
            for start in range(start_id, end_id, batch_size):
                end = min(start + batch_size, end_id)
                urls = [f"{BASE_URL}{date_str}{num:05}" for num in range(start, end)]
                futures = [ex.submit(crawl_url, u) for u in urls]

                for fut in as_completed(futures):
                    res = fut.result()
                    if res:
                        results.append(res)
                        collected += 1
                    pbar.set_postfix_str(f"수집 {collected}")
                    pbar.update(1)
                    time.sleep(sleep_between)

    df = pd.DataFrame(results).drop_duplicates(subset=["url"])
    if save_path:
        df.to_csv(save_path, index=False)
    tqdm.write(f"[{start_id:05}-{end_id-1:05}] 최종 수집: {len(df)}")
    return df

In [6]:
day_str = "20250822"
start_id, end_id = 60000, 80000   # 이 부분만 각 노트북에서 바꿔서 실행
out_path = f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{day_str}_{start_id:05}-{end_id-1:05}.csv"

df_part = crawl_gen_articles_range(
    day_str, start_id, end_id,
    max_workers=36, batch_size=1000, sleep_between=0.05,
    save_path=out_path
)

df_part

20250822 [60000-79999]:  14%|█▍        | 2767/20000 [02:32<15:51, 18.12it/s, 수집 0]


KeyboardInterrupt: 

In [10]:
import requests, pandas as pd, time
from newspaper import Article
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

BASE_URL = "https://www.hankyung.com/article/"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/127.0.0.0 Safari/537.36"),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
    "Referer": "https://www.hankyung.com/",
}

# 세션 재사용 (연결 재활용)
SESSION = requests.Session()
SESSION.headers.update(HEADERS)

def _get_with_retry(url, max_retry=3, timeout=12):
    backoff = 0.7
    for i in range(max_retry):
        r = SESSION.get(url, timeout=timeout)
        # 429/403은 백오프 재시도
        if r.status_code in (429, 403):
            time.sleep(backoff)
            backoff *= 2
            continue
        return r
    return r  # 마지막 응답 반환

def crawl_url(url):
    """AMP 우선, 200이 아니면 일반 페이지 시도. 403/429는 백오프 재시도."""
    try:
        amp = url.rstrip("/") + "/amp"
        r = _get_with_retry(amp)
        if r.status_code != 200:
            r = _get_with_retry(url)  # AMP가 200이 아니면 전부 일반으로 폴백

        if r.status_code not in (200,):
            # 필요시 디버깅: print(f"skip {url} status={r.status_code}")
            return None

        art = Article(url, language='ko')
        art.set_html(r.text)
        art.parse()

        if not art.title or not art.text:
            # 필요시 디버깅: print(f"parse_fail {url}")
            return None

        return {
            "url": url,
            "title": art.title,
            "text": art.text,
            "publish_date": art.publish_date
        }
    except Exception:
        return None

def crawl_gen_articles_range(date_str, start_id, end_id, *,
                             max_workers=24, batch_size=800, sleep_between=0.08,
                             save_path=None):
    results, collected = [], 0
    total = end_id - start_id

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        with tqdm(total=total, desc=f"{date_str} [{start_id:05}-{end_id-1:05}]") as pbar:
            for start in range(start_id, end_id, batch_size):
                end = min(start + batch_size, end_id)
                urls = [f"{BASE_URL}{date_str}{num:05}" for num in range(start, end)]
                futures = [ex.submit(crawl_url, u) for u in urls]

                for fut in as_completed(futures):
                    res = fut.result()
                    if res:
                        results.append(res)
                        collected += 1
                    pbar.set_postfix_str(f"수집 {collected}")
                    pbar.update(1)
                    time.sleep(sleep_between)

    df = pd.DataFrame(results).drop_duplicates(subset=["url"])
    if save_path:
        df.to_csv(save_path, index=False)
    tqdm.write(f"[{start_id:05}-{end_id-1:05}] 최종 수집: {len(df)}")
    return df

In [11]:
day_str = "20250822"
start_id, end_id = 60000, 80000   # 이 부분만 각 노트북에서 바꿔서 실행
out_path = f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{day_str}_{start_id:05}-{end_id-1:05}.csv"

df_part = crawl_gen_articles_range(
    day_str, start_id, end_id,
    max_workers=36, batch_size=1000, sleep_between=0.05,
    save_path=out_path
)

df_part

20250822 [60000-79999]: 100%|██████████| 20000/20000 [48:51<00:00,  6.82it/s, 수집 193]  

[60000-79999] 최종 수집: 193


,url,title,text,publish_date
0,https://www.hankyung.com/article/2025082260104,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
1,https://www.hankyung.com/article/2025082260177,"""그만해 달라"" 외침에도…또래 뺨 때린 '촉법' 중학생 송치",사진=연합뉴스\n\n폭행을 멈춰달라는 부탁에도 또래의 뺨을 계속해서 때린 중학생과 ...,2025-08-22 15:29:09+09:00
2,https://www.hankyung.com/article/2025082260274,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
3,https://www.hankyung.com/article/2025082260324,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
4,https://www.hankyung.com/article/2025082260344,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
...,...,...,...,...
188,https://www.hankyung.com/article/2025082265574,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
189,https://www.hankyung.com/article/2025082265584,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
190,https://www.hankyung.com/article/2025082265667,[속보] 국민의힘 당 대표 후보자 '김문수·장동혁' 결선 진출,"[속보] 특검, 불출석 사유서 낸 김건희에 25일 소환 재통보\n\n김건희 여사가 ...",2025-08-22 17:50:23+09:00
191,https://www.hankyung.com/article/2025082265717,[속보] 국민의힘 대표 본경선서 과반득표자 없어…26일 결선투표,"[속보] 특검, 불출석 사유서 낸 김건희에 25일 소환 재통보\n\n김건희 여사가 ...",2025-08-22 17:50:15+09:00
